<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/Pandas_and_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime
from google.colab import files
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from string import punctuation
punctuation += '«»—–…“”'
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import SGDRegressor

In [0]:
uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)

In [0]:
weather_hist.head()

,date,Summary,weather_type,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


# Самый холодный и самый ветреный дни

In [0]:
weather_hist.loc[weather_hist['temp'].idxmin()]

date                                              2012-02-10 07:00:00+01:00
Summary                                                               Foggy
weather_type                                                           snow
temp                                                               -21.8222
temp_app                                                           -21.8222
humidity                                                                0.8
wind_speed                                                           3.0751
Wind Bearing (degrees)                                                  323
Visibility (km)                                                      1.3685
Loud Cover                                                                0
Pressure (millibars)                                                1033.66
summary                   Foggy starting in the morning continuing until...
Name: 54847, dtype: object

In [0]:
weather_hist.loc[weather_hist['wind_speed'].idxmax()]

date                                              2007-01-29 12:00:00+00:00
Summary                                 Dangerously Windy and Partly Cloudy
weather_type                                                           rain
temp                                                                8.94444
temp_app                                                            3.48333
humidity                                                               0.49
wind_speed                                                          63.8526
Wind Bearing (degrees)                                                  307
Visibility (km)                                                     11.4471
Loud Cover                                                                0
Pressure (millibars)                                                1009.05
summary                   Mostly cloudy throughout the day and windy sta...
Name: 12181, dtype: object

# Дождь летом 2014

Посмотрим на то, какие существуют типы погоды

In [0]:
weather_hist.Summary.unique()

array(['Partly Cloudy', 'Mostly Cloudy', 'Overcast', 'Foggy',
       'Breezy and Mostly Cloudy', 'Clear', 'Breezy and Partly Cloudy',
       'Breezy and Overcast', 'Humid and Mostly Cloudy',
       'Humid and Partly Cloudy', 'Windy and Foggy', 'Windy and Overcast',
       'Breezy and Foggy', 'Windy and Partly Cloudy', 'Breezy',
       'Dry and Partly Cloudy', 'Windy and Mostly Cloudy',
       'Dangerously Windy and Partly Cloudy', 'Dry', 'Windy',
       'Humid and Overcast', 'Light Rain', 'Drizzle', 'Windy and Dry',
       'Dry and Mostly Cloudy', 'Breezy and Dry', 'Rain'], dtype=object)

Будем считать, что дождь соответствует погоде 'Rain', 'Light Rain', 'Drizzle'

In [0]:
weather_hist['date'] =  pd.to_datetime(weather_hist['date'])
weather = weather_hist[['date', 'Summary']]

In [0]:
d1 = datetime.datetime(2014, 6, 1).replace(tzinfo=datetime.timezone.utc)
d2 = datetime.datetime(2014, 8, 30).replace(tzinfo=datetime.timezone.utc)
mask = (weather['date'] >= d1) & (weather['date'] <= d2)
weather = weather.loc[mask]
weather

,date,Summary
70872,2014-08-01 00:00:00+02:00,Overcast
70873,2014-08-01 01:00:00+02:00,Overcast
70874,2014-08-01 02:00:00+02:00,Overcast
70875,2014-08-01 03:00:00+02:00,Mostly Cloudy
70876,2014-08-01 04:00:00+02:00,Partly Cloudy
...,...,...
75232,2014-06-09 19:00:00+02:00,Partly Cloudy
75233,2014-06-09 20:00:00+02:00,Partly Cloudy
75234,2014-06-09 21:00:00+02:00,Partly Cloudy
75235,2014-06-09 22:00:00+02:00,Partly Cloudy


In [0]:
weather = weather.loc[weather['Summary'].isin(['Rain', 'Light Rain', 'Drizzle'])]
weather

,date,Summary


К сожалению, согласно данным, дождь летом 2014 не шел ни разу

# Год, месяц, неделя

Самый ветренный месяц (месяц с наибольшей средней скоростью ветра) - март

In [0]:
weather = weather_hist[['date', 'wind_speed']]
months = weather.groupby(weather.date.dt.month).agg(['mean'])
months.loc[months['wind_speed'].idxmax()]

,wind_speed
,mean
date,
3,13.415825


Самый влажный год (год с наибольшей средней влажностью) - 2005

In [0]:
weather = weather_hist[['date', 'humidity']]
years = weather.groupby(weather.date.dt.year).agg(['mean'])
years.loc[years['humidity'].idxmax()]

,humidity
,mean
date,
2005,0.89


Неделя с самой переменчивой погодой (неделя с наибольшим отклонением температуры) - шестая

In [0]:
weather = weather_hist[['date', 'temp']]
weeks = weather.groupby(weather.date.dt.weekofyear).agg(['std'])
weeks.loc[weeks['temp'].idxmax()]

,temp
,std
date,
6,6.456311


# Признаки текста

In [0]:
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
y_train, y_test = train_test_split(data.deal_probability, random_state=42)
np.random.rand(len(y_test))
np.sqrt(mean_squared_error(y_test, np.random.rand(len(y_test))))

0.5278561191236458

In [0]:
np.full(len(y_test), np.mean(y_train))
np.sqrt(mean_squared_error(y_test, np.full(len(y_test), np.mean(y_train))))

0.26367475849307315

Добавим следующие признаки:

1) Соотношение количества уникальных токенов к общему количеству (процент уникальных) - если в тексте объявления часто повторяются одни и те же слова, то это ухудшает его восприятие

2) Наличие в тексте цифр может указывать на более точное описание параметров и характеристик товара

3) Количество восклицательных знаков - чрезмерное их употребление может отталкивать

4) Количество предложений - текст, разделенный на предложения воспринимается лучше

5) Наличие в тексте английских символов - может указывать на названия фирменных товаров



In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

In [0]:
def tokenizer(text):
  text = text.split()
  text = [txt.strip(punctuation) for txt in text]
  text = [txt.lower() for txt in text if text]
  return text

In [0]:
eng = 'abcdefghijklmnopqrstuvwxyz'
data['number_of_exclam_mark'] = data['description'].apply(lambda x: len([ch for ch in x if ch == '!']))
data['unique_tokens'] = data['description'].apply(lambda x: len(set(tokenizer(x)))/len(tokenizer(x)))
data['number_of_num'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isnumeric()]))
data['number_of_sent'] = data['description'].apply(lambda x: len(x.split('.')))
data['number_of_eng_ch'] = data['description'].apply(lambda x: len([ch for ch in x if ch in eng]))

In [0]:
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 
           'number_of_exclam_mark', 'unique_tokens', 'number_of_eng_ch', 'number_of_num', 'number_of_sent']].values
y = data.deal_probability.values
X.shape

(138715, 9)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Качество улучшилось незначительно - средняя ошибка уменьшилась всего лишь на 0.0001

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2632458527862207

# Векторизация

CountVectorizer:

удалось уменьшить значение ошибки на 0,0005 за счет подбора уровня max_features, min_df и ngram_range
изменение праметра analyzer ухудшало результат

In [0]:
cv = CountVectorizer(max_features=2400, min_df=10, max_df=0.4, ngram_range=(1,1))
X = cv.fit_transform(data.description)
y = data.deal_probability.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = Ridge(alpha=10)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.25069997449245585

TfidfVectorizer:

удалось уменьшить значение ошибки на 0,0018 за счет подбора уровня max_features

изменение праметров analyzer и ngram_range ухудшало результат

In [0]:
cv = TfidfVectorizer(max_features=2700, min_df=5, max_df=0.4)
X = cv.fit_transform(data.description)
y = data.deal_probability.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.246553621292103

SGDRegressor:

удалось улучшить результат с 0.248 до 0.246 за счет увеличения параметров max_iter и n_iter_no_change (но это привело к увеличению времени работы)

изменение параметра alpha влияло незначительно, изменение параметра loss ухудшало результат

In [0]:
cv = CountVectorizer(max_features=2500, min_df=10, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=2500, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

y = data.deal_probability.values

X = hstack([X_cv, X_tfidf])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
regressor = SGDRegressor(max_iter=200, alpha=0.0001, n_iter_no_change = 100)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.24679546244592068